<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 95%; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">Hyperparameters Tuning with Optuna</span></b>
  </div>
</div>



<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-danger">

**Contents** 
</div>

<hr>
  <p><font size="3" face="Arial" font-size="large">
  <ul type="square">

  <li> Basic Optuna concepts;  </li>
  <li> Implementation;  </li>
  <li> Visuals;  </li>
  <li> Pruning;  </li>
  <li> Conslusion;  </li>
  <li> Useful Resources  </li>
  
  </ul>
  </font></p>

</div>

<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* In previous lessons, we looked at finding optimal hyperparameters using classical methods: exhaustive grid search (`GridSearchCV`) and random search from a given distribution (`Random Search`).
* Although `Random Search` significantly speeds up the search process, we might miss a set of hyperparameters where the model performs best.<br>
* And here, an idea might come to mind: "What if we do some guessing at first, like in `Random Search`, and then check more often in those areas where the model showed better accuracy?!" This method is called **Bayesian hyperparameter optimization**.
* The most popular libraries implementing this method are `HyperOpt` and `Optuna`. (in our practice, `HyperOpt` often has failures and unstable performance, so in this notebook we will focus on **`Optuna`**)
</div>

<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-info">

**Key Features of the `Optuna` Framework**
</div>

* Lightweight and highly versatile - suitable for optimizing arbitrary functions and evaluation metrics.
* Incorporates state-of-the-art algorithms specifically adapted for hyperparameter search.
* Supports parallel execution and advanced pruning strategies.
* Includes built-in tools for result visualization.
* Offers seamless integration with many popular libraries and frameworks (e.g., boosting algorithms, **scikit-learn**, **PyTorch**, **Weights & Biases**, among others).

To understand how to use it effectively, we will examine the framework in detail.

</div>


In [1]:
import optuna

import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, train_test_split

<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

**`Optuna` has two core concepts:**


<div class="alert alert-info">

**1. `Study`: an optimization based on an `Objective` function.**
</div>


The `Objective` function should contain the logic for calculating the metric to optimize. Optuna will call this function multiple times to search for the best set of parameters.


<div class="alert" style="background-color:rgb(0, 0, 0); border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color:rgb(255, 255, 255);">

```python
def objective(trial, ...):
    # calculate score...
    return score
```
</div>

<div class="alert alert-info">

**2. `Trial` - a single execution of the `Objective` function**
</div>


Within the `trial` object, we define parameters to be tuned using appropriate methods depending on the type. For example:

<div class="alert" style="background-color:rgb(0, 0, 0); border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color:rgb(255, 255, 255);">

```python
# `suggest_float` method is used to tune float values within the range [0, 1.5]
param = trial.suggest_float('param', 0, 1.5) 

# Categorical value
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Integer value
depth = trial.suggest_int('depth', 5, 8)

# Uniform distribution
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)
```
</div>

</div>

<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
[`Optuna`](https://optuna.readthedocs.io/en/stable/index.html) implements several parameter search methods (`samplers`), including classical ones:
* `GridSampler`
* `RandomSampler`
* `Tree-Structured Parzen Estimator` (`TPESampler` – the most popular and default one)
* `BruteForceSampler`
* And [4 more](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html#module-optuna.samplers); you can also implement a custom sampler.
</div>


<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-info">

**Let's go through a simple example to see how `Optuna` works:**
</div>

* Suppose we have a function `y = (x+1)(x+5)(x-9)`, and we want to find the value of `x` that minimizes the function.
* We'll create an `objective` function and pass a `trial` object as an argument — using it, we'll define the search range for `x` as `[-5, 5]`.

<img src='../imgs/05.2.02_1.JPG' width='600px'>

</div>


In [2]:
# Limit logging
optuna.logging.set_verbosity(30)

In [15]:
def objective(trial):
    x = trial.suggest_float("x", -2, 10)
    return (x + 1) * (x + 5) * (x - 9)


# create a study object and run for 10 iterations; since we are searching for a minimum, the direction parameter is left as default
study = optuna.create_study()

# run the optimization
study.optimize(objective,
               n_jobs=-1,
               n_trials=250,
               show_progress_bar=True)

  0%|          | 0/250 [00:00<?, ?it/s]

In [16]:
optuna.visualization.plot_slice(study)

In [17]:
optuna.visualization.plot_optimization_history(study)

In [18]:
# display best parameters
print("Best parameters:", study.best_params)
# display best value
print("Best value:", study.best_value)

Best parameters: {'x': 5.164410242762302}
Best value: -240.32882810740705


<div class="alert" style="background-color:rgb(255, 255, 255); border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

We see the results of all runs and that the minimum of the function occurred at `x = 5.16`, which is close to the actual minimum.

</div>